## Section 3.1: Extract data from Yahoo! Finance page

 - Load the yfinance module
 - Scan `Yahoo! Finance` page for the stock symbols that appeared in the news
 - Capture/retrieve the `stock price` and `volume` for `last 30 days`.

In [1]:
#Loading yfinance module and installing it if it doesn't already exist
try:
    success_msg = 'Yahoo! Finance module loaded.'
    import yfinance as yf
    print(success_msg)
except ModuleNotFoundError as e:
    print('Installing Yahoo! Finance python module.\n')
    !pip install yfinance
    import yfinance as yf
    print(success_msg)

Yahoo! Finance module loaded.


In [2]:
msft = yf.Ticker("MSFT")
print (msft.info)

{'zip': '98052-6399', 'sector': 'Technology', 'fullTimeEmployees': 181000, 'longBusinessSummary': 'Microsoft Corporation develops, licenses, and supports software, services, devices, and solutions worldwide. Its Productivity and Business Processes segment offers Office, Exchange, SharePoint, Microsoft Teams, Office 365 Security and Compliance, and Skype for Business, as well as related Client Access Licenses (CAL); Skype, Outlook.com, OneDrive, and LinkedIn; and Dynamics 365, a set of cloud-based and on-premises business solutions for organizations and enterprise divisions. Its Intelligent Cloud segment licenses SQL, Windows Servers, Visual Studio, System Center, and related CALs; GitHub that provides a collaboration platform and code hosting service for developers; and Azure, a cloud platform. It also offers support services and Microsoft consulting services to assist customers in developing, deploying, and managing Microsoft server and desktop solutions; and training and certificatio

In [3]:
msft_data=yf.download("MSFT TSLA", start="2021-01-01", end="2021-04-30", group_by="Ticker")
print (msft_data["MSFT"])

[*********************100%***********************]  2 of 2 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2021-01-04  222.529999  223.000000  214.809998  217.690002  215.880417   
2021-01-05  217.259995  218.520004  215.699997  217.899994  216.088684   
2021-01-06  212.169998  216.490005  211.940002  212.250000  210.485626   
2021-01-07  214.039993  219.339996  213.710007  218.289993  216.475418   
2021-01-08  218.679993  220.580002  217.029999  219.619995  217.794373   
...                ...         ...         ...         ...         ...   
2021-04-23  257.880005  261.510010  257.269989  261.149994  259.575653   
2021-04-26  261.660004  262.440002  260.170013  261.549988  259.973236   
2021-04-27  261.579987  263.190002  260.119995  261.970001  260.390686   
2021-04-28  256.079987  256.540009  252.949997  254.559998  253.025360   
2021-04-29  255.460007  256.100006  249.000